# iTelescope premium image set processing, the hard way

This notebook illustrates how to process a premium image set from iTelescope using the AstroPhotography package. The python environment used corresponds to a miniconda emvironment `ap-env.yml`.

The example dataset used is the iTelescope Plan-20 premium dataset of the M101 supernova [SN 2023 ixf](https://en.wikipedia.org/wiki/SN_2023ixf). Note that these files are not provided with this package. However the notebook should work with your own files if you specify a valid fits-file containing directory at the prompt below.

## Notebook environment setup

In [1]:
import os
import pathlib
import sys
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import astropy
from astropy.io import fits
import AstroPhotography as ap

In [2]:
def print_module_version(mod):
    """
    Convenience function to pretty-print Python module mod version.

    Arguments:
        mod {module} -- Imported Python module

    Returns:
        str -- module name and version
    """
    print(f"Using module {mod.__name__} version: {mod.__version__}")
    return

In [3]:
# Get Version information
print(f'Python version: {sys.version}')
print_module_version(np)
print_module_version(matplotlib)
print_module_version(astropy)
print_module_version(ap)

Python version: 3.10.13 | packaged by conda-forge | (main, Dec 23 2023, 15:36:39) [GCC 12.3.0]
Using module numpy version: 1.26.3
Using module matplotlib version: 3.8.2
Using module astropy version: 6.0.0
Using module AstroPhotography version: 0.5.2


In [4]:
# Enable inline plotting for graphics
# %matplotlib inline
# Set default figure size to be larger
# this may only work in matplotlib 2.0+!
from IPython.core.interactiveshell import InteractiveShell
matplotlib.rcParams['figure.figsize'] = [10.0, 6.0]
# Enable multiple outputs from jupyter cells
InteractiveShell.ast_node_interactivity = "all"

## What files do we have to work with?

Let us change the working directory from the location of this notebook to a directory holding some iTelescope premium image set files. In my case this is the Plan-20 M101 dataset.

**Note:** The following processing is fairly low-level and requires a lot of human interaction. In a later section we'll process the same set of files in a more automated way.

In [5]:
print('Enter the path to the directory containing the premium image set')
wdir = input('Image set path:')
os.chdir(wdir)
print('Switched directory to ' + os.getcwd())

Enter the path to the directory containing the premium image set


Image set path: /old_lnx/home/dks/Downloads/iTelescopeScratch/Plan-20-M101SN


Switched directory to /old_lnx/home/dks/Downloads/iTelescopeScratch/Plan-20-M101SN


We now want a pattern that can be used to select the FITS files we want to process, while ignoring any other files or directories we have in that directory (e.g. processed files, configurations files, and so on).

Listing the fits files from the shell I get:
```bash
ls ../Plan-20-M101SN/*fits
../Plan-20-M101SN/Calibrated-iTelescope-M101_Supernova_2023ixf-180s-Lum-1.fits
../Plan-20-M101SN/Calibrated-iTelescope-M101_Supernova_2023ixf-180s-Lum-2.fits
../Plan-20-M101SN/Calibrated-iTelescope-M101_Supernova_2023ixf-180s-Lum-3.fits
../Plan-20-M101SN/Calibrated-iTelescope-M101_Supernova_2023ixf-300s-Blue-1.fits
../Plan-20-M101SN/Calibrated-iTelescope-M101_Supernova_2023ixf-300s-Blue-2.fits
../Plan-20-M101SN/Calibrated-iTelescope-M101_Supernova_2023ixf-300s-Blue-3.fits
../Plan-20-M101SN/Calibrated-iTelescope-M101_Supernova_2023ixf-300s-Green-1.fits
../Plan-20-M101SN/Calibrated-iTelescope-M101_Supernova_2023ixf-300s-Green-2.fits
../Plan-20-M101SN/Calibrated-iTelescope-M101_Supernova_2023ixf-300s-Green-3.fits
../Plan-20-M101SN/Calibrated-iTelescope-M101_Supernova_2023ixf-300s-Red-1.fits
../Plan-20-M101SN/Calibrated-iTelescope-M101_Supernova_2023ixf-300s-Red-2.fits
../Plan-20-M101SN/Calibrated-iTelescope-M101_Supernova_2023ixf-300s-Red-3.fits
```

So a valid pattern that picks up the original files is `Calibrated-iTelescope*.fits`

In [6]:
default_pattern = 'Calibrated-iTelescope*.fits'
print(f'Default file pattern for input files: {default_pattern}')
msg = 'Enter new input file pattern (or return to accept default pattern)'
file_pattern = input(msg).strip() or default_pattern
print(f'Using "{file_pattern}" as the input file pattern.')

Default file pattern for input files: Calibrated-iTelescope*.fits


Enter new input file pattern (or return to accept default pattern) 


Using "Calibrated-iTelescope*.fits" as the input file pattern.


In [7]:
p_dir = pathlib.Path(r'./')
flist = list(sorted(p_dir.glob(file_pattern)))
print(f'{len(flist)} files match pattern "{file_pattern}" in current directory.')
for fpath in flist:
    print(f'  {fpath.name}')

12 files match pattern "Calibrated-iTelescope*.fits" in current directory.
  Calibrated-iTelescope-M101_Supernova_2023ixf-180s-Lum-1.fits
  Calibrated-iTelescope-M101_Supernova_2023ixf-180s-Lum-2.fits
  Calibrated-iTelescope-M101_Supernova_2023ixf-180s-Lum-3.fits
  Calibrated-iTelescope-M101_Supernova_2023ixf-300s-Blue-1.fits
  Calibrated-iTelescope-M101_Supernova_2023ixf-300s-Blue-2.fits
  Calibrated-iTelescope-M101_Supernova_2023ixf-300s-Blue-3.fits
  Calibrated-iTelescope-M101_Supernova_2023ixf-300s-Green-1.fits
  Calibrated-iTelescope-M101_Supernova_2023ixf-300s-Green-2.fits
  Calibrated-iTelescope-M101_Supernova_2023ixf-300s-Green-3.fits
  Calibrated-iTelescope-M101_Supernova_2023ixf-300s-Red-1.fits
  Calibrated-iTelescope-M101_Supernova_2023ixf-300s-Red-2.fits
  Calibrated-iTelescope-M101_Supernova_2023ixf-300s-Red-3.fits


### Do these have full headers?

Lets have a look at the FITS header for the first file in this list.

In [8]:
hdr = None
with fits.open(flist[0], mode='readonly') as hdulist:
    print(f'Opened {flist[0].name}')
    hdulist.verify('fix')
    print(hdulist.info())
    hdr = hdulist[0].header
    print(f'Printing the FITS header, length={len(hdr)}')
    hdr

Opened Calibrated-iTelescope-M101_Supernova_2023ixf-180s-Lum-1.fits
Filename: Calibrated-iTelescope-M101_Supernova_2023ixf-180s-Lum-1.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      17   (3056, 3056)   float32   
None
Printing the FITS header, length=17


SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -32 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                 3056                                                  
NAXIS2  =                 3056                                                  
EXTEND  =                    T                                                  
OWNER   = 'iTelescope'                                                          
RA      = '14 03 12.40'        / [hms J2000] Target right ascension             
OBJCTRA = '14 03 12.40'        / [hms J2000] Target right ascension             
DEC     = '+54 20 58.0'        / [dms +N J2000] Target declination              
OBJCTDEC= '+54 20 58.0'        / [dms +N J2000] Target declination              
FOCALLEN=               3962.0 / Focal length of telescope in mm                
APTDIA  =                610

In [9]:
# What keywords do we have (non-comment, non-history)
sorted(list( hdr.keys() ))

['APTDIA',
 'BITPIX',
 'DEC',
 'EXPTIME',
 'EXTEND',
 'FILTER',
 'FOCALLEN',
 'NAXIS',
 'NAXIS1',
 'NAXIS2',
 'OBJCTDEC',
 'OBJCTRA',
 'OBSTARGET',
 'OWNER',
 'RA',
 'SIMPLE',
 'SRCLINK']

Lets look at the keyword differences in more detail

```python
def fkeywords(fname):
    with fits.open(fname) as hdulist:
        hdr=hdulist[0].header
        print(f'FITS header for primary HDU of {fname}')
        print( sorted( list( hdr.keys() ) ) )
    return

f1='Calibrated-iTelescope-M101_Supernova_2023ixf-180s-Lum-1.fits'
```

On the M101 files we've been looking at:
```python
fkeywords(f1)
FITS header for primary HDU of Calibrated-iTelescope-M101_Supernova_2023ixf-180s-Lum-1.fits
['APTDIA', 'BITPIX', 'DEC', 'EXPTIME', 'EXTEND', 'FILTER', 'FOCALLEN', 'NAXIS', 'NAXIS1', 'NAXIS2', 'OBJCTDEC', 'OBJCTRA', 'OBSTARGET', 'OWNER', 'RA', 'SIMPLE', 'SRCLINK']

# iTelescope calibration system
In [17]: f2_itel='calibrated-T05-davestrickland-M82-20210219-002600-Red-BIN1-W-300-001.fit'

# iTelescope raw file
In [18]: f2_raw='raw-T05-davestrickland-M82-20210219-002600-Red-BIN1-W-300-001.fit'

# Calibrated file produced by AstroPhotography from the raw file above...
In [19]: f2_apcal='cal-T05-davestrickland-M82-20210219-002600-Red-BIN1-W-300-001.fits'

In [20]: fkeywords(f2_raw)
FITS header for primary HDU of raw-T05-davestrickland-M82-20210219-002600-Red-BIN1-W-300-001.fit
['APTAREA', 'APTDIA', 'BITPIX', 'BSCALE', 'BZERO', 'CBLACK', 'CCD-TEMP', 'CSTRETCH', 'CWHITE', 'DATE-OBS', 'EGAIN', 'EXPOSURE', 'EXPTIME', 'FILTER', 'FLIPSTAT', 'FOCALLEN', 'IMAGETYP', 'INSTRUME', 'JD', 'JD-HELIO', 'NAXIS', 'NAXIS1', 'NAXIS2', 'NOTES', 'OBJECT', 'OBSERVER', 'PEDESTAL', 'READOUTM', 'ROWORDER', 'SBSTDVER', 'SET-TEMP', 'SIMPLE', 'SITELAT', 'SITELONG', 'SWCREATE', 'SWOWNER', 'SWSERIAL', 'TELESCOP', 'TRAKTIME', 'XBINNING', 'XORGSUBF', 'XPIXSZ', 'YBINNING', 'YORGSUBF', 'YPIXSZ']

In [21]: fkeywords(f2_itel)
FITS header for primary HDU of calibrated-T05-davestrickland-M82-20210219-002600-Red-BIN1-W-300-001.fit
['AIRMASS', 'ALT-OBS', 'APTAREA', 'APTDIA', 'BITPIX', 'BSCALE', 'BZERO', 'CALSTAT', 'CBLACK', 'CCD-TEMP', 'CSTRETCH', 'CWHITE', 'DATE', 'DATE-HP', 'DATE-OBS', 'DEC', 'EGAIN', 'EXPOSURE', 'EXPTIME', 'FILTER', 'FLIPSTAT', 'FOCALLEN', 'HISTORY', 'HISTORY', 'HISTORY', 'HISTORY', 'HISTORY', 'HISTORY', 'HISTORY', 'IMAGETYP', 'INSTRUME', 'JD', 'JD-HELIO', 'LAT-OBS', 'LONG-OBS', 'NAXIS', 'NAXIS1', 'NAXIS2', 'NOTES', 'OBJCTDEC', 'OBJCTRA', 'OBJECT', 'OBSERVAT', 'OBSERVER', 'PEDESTAL', 'PIERSIDE', 'RA', 'RADECSYS', 'READOUTM', 'ROWORDER', 'RollAngle', 'SBSTDVER', 'SET-TEMP', 'SIMPLE', 'SITELAT', 'SITELONG', 'ST', 'SWCREATE', 'SWMODIFY', 'SWOWNER', 'SWSERIAL', 'TELESCOP', 'TIME-OBS', 'TIMESYS', 'TRAKTIME', 'USERNAME', 'UT', 'XBINNING', 'XORGSUBF', 'XPIXSZ', 'YBINNING', 'YORGSUBF', 'YPIXSZ', 'iTelescope', 'iTelescopePlateScaleH', 'iTelescopePlateScaleV']

In [22]: fkeywords(f2_apcal)
FITS header for primary HDU of cal-T05-davestrickland-M82-20210219-002600-Red-BIN1-W-300-001.fits
['AIRMASS', 'ALT-OBS', 'APTAREA', 'APTDIA', 'BIASCORR', 'BIASFILE', 'BITPIX', 'BPIXCORR', 'BPIXDPIX', 'BPIXFILE', 'BPIXNBAD', 'BPIXNFIX', 'BPIXNREM', 'BPIX_MIN', 'BUNIT', 'CBLACK', 'CCD-TEMP', 'CR_CLEAN', 'CR_NPIX', 'CSTRETCH', 'CWHITE', 'DARKCORR', 'DARKFILE', 'DATE-OBS', 'DEC-OBJ', 'EGAIN', 'EXPOSURE', 'EXPTIME', 'FILTER', 'FLATCORR', 'FLATFILE', 'FLIPSTAT', 'FOCALLEN', 'HISTORY', 'HISTORY', 'HISTORY', 'HISTORY', 'HISTORY', 'HISTORY', 'IMAGETYP', 'INSTRUME', 'JD', 'JD-HELIO', 'LAT-OBS', 'LON-OBS', 'NAXIS', 'NAXIS1', 'NAXIS2', 'NOTES', 'OBJECT', 'OBJNAME', 'OBSERVAT', 'OBSERVER', 'RA-OBJ', 'READOUTM', 'ROWORDER', 'SBSTDVER', 'SET-TEMP', 'SIMPLE', 'SITELAT', 'SITELONG', 'SWCREATE', 'SWOWNER', 'SWSERIAL', 'TELESCOP', 'TRAKTIME', 'XBINNING', 'XORGSUBF', 'XPIXSZ', 'YBINNING', 'YORGSUBF', 'YPIXSZ']

```

After some rearranging by hand we get the following table of header keywords. The premium file is clearly missing a large number of header keywords, although this doesn't tell us which ones are needed and which ones are superfluous.

|iTelescope raw|iTelescope calibrated|AP calibrated|M101 premium|
|--------------|---------------------|-------------|------------|
|              |AIRMASS              |AIRMASS      |            |
|              |ALT-OBS              |ALT-OBS      |            |
|APTAREA       |APTAREA              |APTAREA      |            |
|APTDIA        |APTDIA               |APTDIA       |APTDIA      |
|              |                     |BIASCORR     |            |
|              |                     |BIASFILE     |            |
|BITPIX        |BITPIX               |BITPIX       |BITPIX      |
|              |                     |BPIXCORR     |            |
|              |                     |BPIXDPIX     |            |
|              |                     |BPIXFILE     |            |
|              |                     |BPIXNBAD     |            |
|              |                     |BPIXNFIX     |            |
|              |                     |BPIXNREM     |            |
|              |                     |BPIX_MIN     |            |
|BSCALE        |BSCALE               |             |            |
|BZERO         |BZERO                |             |            |
|              |                     |BUNIT        |            |
|              |CALSTAT              |             |            |
|CBLACK        |CBLACK               |CBLACK       |            |
|CCD-TEMP      |CCD-TEMP             |CCD-TEMP     |            |
|              |                     |CR_CLEAN     |            |
|              |                     |CR_NPIX      |            |
|CSTRETCH      |CSTRETCH             |CSTRETCH     |            |
|CWHITE        |CWHITE               |CWHITE       |            |
|              |                     |DARKCORR     |            |
|              |                     |DARKFILE     |            |
|              |DATE                 |             |            |
|              |DATE-HP              |             |            |
|DATE-OBS      |DATE-OBS             |DATE-OBS     |            |
|              |DEC                  |             |DEC         |
|              |                     |DEC-OBJ      |            |
|EGAIN         |EGAIN                |EGAIN        |            |
|EXPOSURE      |EXPOSURE             |EXPOSURE     |            |
|EXPTIME       |EXPTIME              |EXPTIME      |EXPTIME     |
|              |                     |             |EXTEND      |
|FILTER        |FILTER               |FILTER       |FILTER      |
|              |                     |FLATCORR     |            |
|              |                     |FLATFILE     |            |
|FLIPSTAT      |FLIPSTAT             |FLIPSTAT     |            |
|FOCALLEN      |FOCALLEN             |FOCALLEN     |FOCALLEN    |
|IMAGETYP      |IMAGETYP             |IMAGETYP     |            |
|INSTRUME      |INSTRUME             |INSTRUME     |            |
|JD            |JD                   |JD           |            |
|JD-HELIO      |JD-HELIO             |JD-HELIO     |            |
|              |LAT-OBS              |LAT-OBS      |            |
|              |LONG-OBS             |LON-OBS      |            |
|NAXIS         |NAXIS                |NAXIS        |NAXIS       |
|NAXIS1        |NAXIS1               |NAXIS1       |NAXIS1      |
|NAXIS2        |NAXIS2               |NAXIS2       |NAXIS2      |
|NOTES         |NOTES                |NOTES        |            |
|              |OBJCTDEC             |             |OBJCTDEC    |
|              |OBJCTRA              |             |OBJCTRA     |
|OBJECT        |OBJECT               |OBJECT       |            |
|              |                     |OBJNAME      |            |
|              |OBSERVAT             |OBSERVAT     |            |
|OBSERVER      |OBSERVER             |OBSERVER     |            |
|              |                     |             |OBSTARGET   |
|              |                     |             |OWNER       |
|PEDESTAL      |PEDESTAL             |             |            |
|              |PIERSIDE             |             |            |
|              |RA                   |             |RA          |
|              |RADECSYS             |             |            |
|              |                     |RA-OBJ       |            |
|READOUTM      |READOUTM             |READOUTM     |            |
|ROWORDER      |ROWORDER             |ROWORDER     |            |
|              |RollAngle            |             |            |
|SBSTDVER      |SBSTDVER             |SBSTDVER     |            |
|SET-TEMP      |SET-TEMP             |SET-TEMP     |            |
|SIMPLE        |SIMPLE               |SIMPLE       |SIMPLE      |
|SITELAT       |SITELAT              |SITELAT      |            |
|SITELONG      |SITELONG             |SITELONG     |            |
|              |                     |             |SRCLINK     |
|              |ST                   |             |            |
|SWCREATE      |SWCREATE             |SWCREATE     |            |
|SWOWNER       |SWMODIFY             |SWOWNER      |            |
|SWSERIAL      |SWOWNER              |SWSERIAL     |            |
|              |SWSERIAL             |             |            |
|TELESCOP      |TELESCOP             |TELESCOP     |            |
|              |TIME-OBS             |             |            |
|              |TIMESYS              |             |            |
|TRAKTIME      |TRAKTIME             |TRAKTIME     |            |
|              |USERNAME             |             |            |
|              |UT                   |             |            |
|XBINNING      |XBINNING             |XBINNING     |            |
|XORGSUBF      |XORGSUBF             |XORGSUBF     |            |
|XPIXSZ        |XPIXSZ               |XPIXSZ       |            |
|YBINNING      |YBINNING             |YBINNING     |            |
|YORGSUBF      |YORGSUBF             |YORGSUBF     |            |
|YPIXSZ        |YPIXSZ               |YPIXSZ       |            |
|              |iTelescope           |             |            |
|              |iTelescopePlateScaleH|             |            |
|              |iTelescopePlateScaleV|             |            |


## Astrometry (aka Navigation)

Combining multiple images of the same source in the same filter requires that each image have a valid World Coordinate System solution, based on an astrometric solution to the stars in the field of view. The `AstroPhotography` package also refers to this process as image navigation. 

The process consists of finding the pixel locations brightest N stars in each image, then calling `Astrometry.net` to compute a WCS solution based on those star locations, and creating a new "navigated" fits file that combines the calibrated file and the WCS solution. By default 2-dimensional Gaussian profiles are also fitted to a representative number of stars per image to assess the Full Width at Half Maximum of the star images (in pixels), the results of which are shown graphically in a plot and numerically in a YaML file describing the stars found and fitted per image. At the end of the process a summary of the image (star) statistics is generated in CSV format that also incorporates the plate scale (arcseconds per pixel) found in the astrometric solution, which allows a human to look for outliers such as images where the seeing or tracking was especially bad.

In terms of pseudo-code
```python
for cal-img in calibrated-images:
    ap_find_stars cal-img --> source-list source-plot fwhm-plot quality-report.yml ds9-sources.reg
    ap_astrometry cal-img source-list --> navigated-img
ap_quality_summary (all quality-report.yml files) --> quality-summary.csv
```
Using the (current but deprecated) bash script `navigate_all.sh` the output for a simple calibrated file is as follows:
```bash
Processing ./20210306/cal-T05-davestrickland-M81-20210306-235058-B-BIN1-W-300-001.fits at Sun Feb 11 04:17:23 PM EST 2024
  Cleaning all files...
removed 'SourceLists/srclist_T05-davestrickland-M81-20210306-235058-B-BIN1-W-300-001.fits'
removed 'SourceLists/implot_T05-davestrickland-M81-20210306-235058-B-BIN1-W-300-001.png'
removed 'SourceLists/ds9_T05-davestrickland-M81-20210306-235058-B-BIN1-W-300-001.reg'
removed 'MetaData/qual_T05-davestrickland-M81-20210306-235058-B-BIN1-W-300-001.yaml'
removed 'MetaData/srclog_T05-davestrickland-M81-20210306-235058-B-BIN1-W-300-001.log'
removed 'NavigatedImages/nav_T05-davestrickland-M81-20210306-235058-B-BIN1-W-300-001.fits'
  Performing star detection on ./20210306/cal-T05-davestrickland-M81-20210306-235058-B-BIN1-W-300-001.fits
    About the run the following command python3 /home/dks/git/AstroPhotography/AstroPhotography/scripts/ap_find_stars.py -l DEBUG -m 200 ./20210306/cal-T05-davestrickland-M81-20210306-235058-B-BIN1-W-300-001.fits SourceLists/srclist_T05-davestrickland-M81-20210306-235058-B-BIN1-W-300-001.fits --retain_saturated --plotfile=SourceLists/implot_T05-davestrickland-M81-20210306-235058-B-BIN1-W-300-001.png --fwhm_plot=SourceLists/fwhmplot_T05-davestrickland-M81-20210306-235058-B-BIN1-W-300-001.png --quality_report=MetaData/qual_T05-davestrickland-M81-20210306-235058-B-BIN1-W-300-001.yaml --ds9=SourceLists/ds9_T05-davestrickland-M81-20210306-235058-B-BIN1-W-300-001.reg
    Star detection completed successfully at Sun Feb 11 04:17:28 PM EST 2024
      Log file written to MetaData/srclog_T05-davestrickland-M81-20210306-235058-B-BIN1-W-300-001.log
  Performing astrometry on ./20210306/cal-T05-davestrickland-M81-20210306-235058-B-BIN1-W-300-001.fits
    About the run the following command:  python3 /home/dks/git/AstroPhotography/AstroPhotography/scripts/ap_astrometry.py -l DEBUG ./20210306/cal-T05-davestrickland-M81-20210306-235058-B-BIN1-W-300-001.fits SourceLists/srclist_T05-davestrickland-M81-20210306-235058-B-BIN1-W-300-001.fits NavigatedImages/nav_T05-davestrickland-M81-20210306-235058-B-BIN1-W-300-001.fits
    Astrometry completed successfully at Sun Feb 11 04:17:38 PM EST 2024
```

### Navigating a single image

In [19]:
p_loglevel  = 'DEBUG'                                                     # Loglevel
p_fitsimg   = str(flist[0])                                               # Input fits image
p_extnum    = 0                                                           # Extension number for data in input fits
p_fitstbl   = p_fitsimg.replace('Calibrated-iTelescope', 'srclist')       # Output fits table of srcs
p_search_fwhm       = 3.0                                                 # initial estimate at star fwhm in pixels
p_search_nsigma     = 7.0                                                 # min detection sigma above background
p_detector_bitdepth = 16                                                  # detector bitdepth (16 for most CCDs, ?? for CMOS, ?? for camera)
p_sat_frac          = 0.8                                                 # Fraction of full well at which we assume star saturated
p_max_sources = 200                                                       # Max number of sources to output or None
p_nosatmask   = True                                                      # Keep possibly saturated stars.
p_regfile     = p_fitsimg.replace('Calibrated-iTelescope', 'ds9').replace('fits', 'reg')       # Name of ds9 region file or None
p_plotfile    = p_fitsimg.replace('Calibrated-iTelescope', 'implot').replace('fits', 'png')    # None or name of output png image w sources.
p_qual_rprt   = p_fitsimg.replace('Calibrated-iTelescope', 'qual').replace('fits', 'yaml')     # None or name of output ascii report
p_quiet       = False                                                                          # Quiet mode suppresses STDOUT list printing
p_fwhm_plot   = p_fitsimg.replace('Calibrated-iTelescope', 'fwhmplot').replace('fits', 'png')  # PNG/JPG plot of PSF fits